In [16]:
import requests
import pandas as pd
import hopsworks
import datetime
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util
sys.path.append(os.path.abspath(os.path.join('..', 'model')))
import nn
import torch
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

In [17]:
project = hopsworks.login()
fs = project.get_feature_store() 

2025-01-03 14:51:52,594 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-03 14:51:52,606 INFO: Initializing external client
2025-01-03 14:51:52,606 INFO: Base URL: https://c.app.hopsworks.ai:443

Multiple projects found. 

	 (1) KTH_ID2223
	 (2) simon003
2025-01-03 14:51:56,157 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164449


In [25]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="price_prediction_model",
    version=4,
)

saved_model_dir = retrieved_model.download()

In [26]:
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='6.909423E-1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
today = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
yesterday = today - timedelta(days=1)

In [37]:
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
power_fg = fs.get_feature_group(
    name='power',
    version=1,
)

el_prices_fg = fs.get_feature_group(
    name='el_prices',
    version=1,
)


weather_batch_data = weather_fg.read()
power_batch_data = power_fg.read()
el_prices_data = el_prices_fg.read()


batch_data = weather_batch_data.merge(power_batch_data, on="date", how="inner")
batch_data['date'] = pd.to_datetime(batch_data['date'])
batch_data = batch_data.sort_values(by="date", ascending=False).reset_index(drop=True)


el_prices_data['date'] = pd.to_datetime(el_prices_data['date'])
el_prices_data = el_prices_data.sort_values(by="date", ascending=True).reset_index(drop=True)

el_prices_data['price_lag_1'] = el_prices_data['price'].shift(1)
el_prices_data['price_lag_2'] = el_prices_data['price'].shift(2)
el_prices_data['price_lag_3'] = el_prices_data['price'].shift(3)

el_prices_data = el_prices_data.dropna().reset_index(drop=True)

batch_data_with_lags = batch_data.merge(el_prices_data, on="date", how="inner")

latest_data = batch_data_with_lags.iloc[0]
latest_data = latest_data.drop('price')
latest_data = latest_data.drop('date')
print(latest_data)
latest_data = latest_data.apply(pd.to_numeric, errors='coerce')
print(latest_data)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.65s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
temperature_2m_mean               -4.347083
precipitation_sum                       0.0
wind_speed_10m_max                21.959999
wind_direction_10m_dominant      283.490631
sunshine_duration              14318.928711
hydro_mw                            9434.75
nuclear_mw                           5827.5
other_mw                        1057.208333
wind_mw                         6998.458333
price_lag_1                        0.075346
price_lag_2                        0.324242
price_lag_3                          0.2303
Name: 0, dtype: object
temperature_2m_mean               -4.347083
precipitation_sum                  0.000000
wind_speed_10m_max                21.959999
wind_direction_10m_dominant      283.490631
sunshine_durati

In [43]:
print(latest_data[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant', 'sunshine_duration', 'hydro_mw', 'nuclear_mw', 'other_mw', 'wind_mw', 'price_lag_1', 'price_lag_2', 'price_lag_3']])

temperature_2m_mean               -4.347083
precipitation_sum                  0.000000
wind_speed_10m_max                21.959999
wind_direction_10m_dominant      283.490631
sunshine_duration              14318.928711
hydro_mw                        9434.750000
nuclear_mw                      5827.500000
other_mw                        1057.208333
wind_mw                         6998.458333
price_lag_1                        0.075346
price_lag_2                        0.324242
price_lag_3                        0.230300
Name: 0, dtype: float64


In [49]:
latest_data_df = latest_data[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 
                              'wind_direction_10m_dominant', 'sunshine_duration', 'hydro_mw', 
                              'nuclear_mw', 'other_mw', 'wind_mw', 'price_lag_1', 'price_lag_2', 'price_lag_3']]

latest_data_df = latest_data_df.values.reshape(1, -1)

prediction = retrieved_xgboost_model.predict(latest_data_df)[0]

print(prediction)

0.6953426


Upload value to hopswork